# Ejercicio 1: Introducción a Recuperación de Información

JORGE ROJAS

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un conjunto de documentos de texto.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [1]:
archivos = ['01_corpus_turismo_500.txt', '01_corpus_turismo.txt']
palabra = input("Palabra a buscar: ")

documentos_encontrados = []

for archivo in archivos:
    with open(archivo, 'r', encoding='utf-8') as f:
        contenido = f.read()
        if palabra.lower() in contenido.lower():
            documentos_encontrados.append(archivo)

if documentos_encontrados:
    print(f"La palabra '{palabra}' se encontró en los siguientes documentos:")
    for doc in documentos_encontrados:
        print(f"- {doc}")
else:
    print(f"La palabra '{palabra}' no se encontró en ningún documento.")

La palabra 'Ecuador' se encontró en los siguientes documentos:
- 01_corpus_turismo_500.txt
- 01_corpus_turismo.txt


## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

In [2]:
archivos = ['01_corpus_turismo_500.txt', '01_corpus_turismo.txt']
indice_invertido = {}

for archivo in archivos:
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            for num_linea, linea in enumerate(f, start=1):
                for palabra in linea.strip().lower().split():
                    indice_invertido.setdefault(palabra, {}).setdefault(archivo, []).append(num_linea)
    except FileNotFoundError:
        print(f"Archivo no encontrado: {archivo}")

consultas = input("Palabras a buscar (separadas por espacios): ").strip().lower().split()
resultados = {consulta: indice_invertido.get(consulta, {}) for consulta in consultas}

for consulta, docs in resultados.items():
    if docs:
        print(f"\nLa palabra '{consulta}' se encontró en:")
        for archivo, lineas in docs.items():
            print(f"- {archivo}: líneas: {', '.join(map(str, lineas))}")
    else:
        print(f"\nLa palabra '{consulta}' no se encontró en ningún documento.")


La palabra 'ecuador' se encontró en:
- 01_corpus_turismo_500.txt: líneas: 5, 22, 30, 37, 41, 45, 76, 109, 110, 122, 127, 130, 156, 163, 168, 200, 225, 231, 246, 254, 258, 280, 307, 319, 324, 333, 352, 371, 379, 383, 386, 442, 459, 486, 491, 492
- 01_corpus_turismo.txt: líneas: 1

La palabra 'quito' se encontró en:
- 01_corpus_turismo_500.txt: líneas: 4, 16, 17, 39, 43, 73, 75, 89, 104, 136, 201, 213, 238, 260, 291, 325, 355, 391, 469, 487
- 01_corpus_turismo.txt: líneas: 3


## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño: 16 documentos (turismo en Ecuador).
      -  Corpus grande: 500 documentos (versión ampliada).
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

### Ejemplo de palabras para buscar
- quito
- montañita
- feriado
- playas
- aventura
- galápagos

In [3]:
import time

corpus_pequeno = ['01_corpus_turismo.txt']
corpus_grande = ['01_corpus_turismo_500.txt']
palabras_buscar = ['quito', 'montañita', 'feriado', 'playas', 'aventura', 'galápagos']

def busqueda_lineal(archivos, palabras):
    return {palabra: {archivo: [num for num, linea in enumerate(open(archivo, 'r', encoding='utf-8'), start=1) 
                                if palabra in linea.lower()] for archivo in archivos if palabra in open(archivo, 'r', encoding='utf-8').read().lower()} 
            for palabra in palabras}

def construir_indice(corpus):
    indice = {}
    for archivo in archivos:
        with open(archivo, 'r', encoding='utf-8') as f:
            for num, linea in enumerate(f, start=1):
                for palabra in linea.strip().lower().split():
                    indice.setdefault(palabra, {}).setdefault(archivo, []).append(num)
    return indice

def busqueda_indice(indice, palabras):
    return {palabra: indice.get(palabra, {}) for palabra in palabras}

def medir_tiempos(archivos, palabras):
    inicio = time.time()
    busqueda_lineal(archivos, palabras)
    tiempo_lineal = time.time() - inicio

    inicio = time.time()
    indice = construir_indice(archivos)
    tiempo_construccion = time.time() - inicio

    inicio = time.time()
    busqueda_indice(indice, palabras)
    tiempo_busqueda_indice = time.time() - inicio

    return tiempo_lineal, tiempo_construccion, tiempo_busqueda_indice

for corpus, archivos in [("Corpus Pequeño", corpus_pequeno), ("Corpus Grande", corpus_grande)]:
    tiempos = medir_tiempos(archivos, palabras_buscar)
    print(f"{corpus}: Búsqueda Lineal={tiempos[0]:.4f}s, Construcción Índice={tiempos[1]:.4f}s, Búsqueda Índice={tiempos[2]:.4f}s")

Corpus Pequeño: Búsqueda Lineal=0.0015s, Construcción Índice=0.0002s, Búsqueda Índice=0.0000s
Corpus Grande: Búsqueda Lineal=0.0062s, Construcción Índice=0.0016s, Búsqueda Índice=0.0000s


## Parte 4:
### Actividad
1. Modifica el índice para que ignore mayúsculas/minúsculas (por ejemplo, "Playa" y "playa" deben considerarse iguales).
2. Permite consultas de múltiples términos (ejemplo: buscar documentos que contengan "playa" y "turismo").
3. Calcula el _speedup_

In [4]:
import time

corpus_pequeno = ['01_corpus_turismo.txt']
corpus_grande = ['01_corpus_turismo_500.txt']
palabras_buscar = ['quito', 'montañita', 'feriado', 'playas', 'aventura', 'galápagos']

def construir_indice(archivos):
    indice = {}
    for archivo in archivos:
        with open(archivo, 'r', encoding='utf-8') as f:
            for num, linea in enumerate(f, 1):
                for palabra in linea.strip().lower().split():
                    indice.setdefault(palabra, {}).setdefault(archivo, []).append(num)
    return indice

def busqueda_indice(indice, palabras):
    return {archivo: set(sum((indice.get(p, {}).get(archivo, []) for p in palabras), [])) 
            for p in palabras for archivo in indice.get(p, {})}

def busqueda_lineal(archivos, palabras):
    return {archivo: [num for num, linea in enumerate(open(archivo, 'r', encoding='utf-8'), 1) 
                      if all(p in linea.lower() for p in palabras)] for archivo in archivos}

def medir_tiempos(archivos, palabras):
    t1 = time.time(); busqueda_lineal(archivos, palabras); t_lineal = time.time() - t1
    t2 = time.time(); indice = construir_indice(archivos); t_indice = time.time() - t2
    t3 = time.time(); busqueda_indice(indice, palabras); t_busqueda = time.time() - t3
    return t_lineal, t_indice, t_busqueda, t_lineal / (t_indice + t_busqueda)

for corpus, archivos in [("Corpuss Pequeño", corpus_pequeno), ("Corpus Grande", corpus_grande)]:
    t_lineal, t_indice, t_busqueda, speedup = medir_tiempos(archivos, palabras_buscar)
    print(f"{corpus}: Lineal={t_lineal:.4f}s, Índice={t_indice:.4f}s, Búsqueda={t_busqueda:.4f}s, Speedup={speedup:.2f}x")

Corpuss Pequeño: Lineal=0.0006s, Índice=0.0003s, Búsqueda=0.0000s, Speedup=1.65x
Corpus Grande: Lineal=0.0020s, Índice=0.0032s, Búsqueda=0.0001s, Speedup=0.62x
